In [2]:
import disney_parks

In [1]:
host = "disneydbinstance.c9lxpmnhtw8a.us-east-2.rds.amazonaws.com"
dbname = "DisneyDB"
username = "chrisArnold1228"
password = "Simba49ers!"
port = 3306
weatherAPI = "686f48c707798f004bf424ed785b2c27"

In [3]:
import pymysql
import re
from datetime import datetime
from pytz import timezone
import pandas as pd
tz = timezone('US/Eastern')

In [ ]:
park_ids = open("parkIds.txt", "r")
string = park_ids.read()
park_list = str.splitlines(string)
ids = [x.split(':')[1] for x in park_list]

conn = pymysql.connect(host, user=username,port=port,passwd=password)
cur = conn.cursor()

for current_id in ids:
    #print(current_id)
    park = disney_parks.Park(current_id)
    current_park_name = park.getParkName().replace("'","")
    query = "select * from DisneyDB.Park where Name = '" + current_park_name + "'"
    park_table = pd.read_sql_query(query, conn)
    if park_table.shape[0] < 1:
        insert_park = "insert into DisneyDB.Park (Name) values ('%s')" %(current_park_name)
        cur.execute(insert_park)
        conn.commit()
        park_table = pd.read_sql_query(query, conn)

    park_db_id = park_table['Id'][0]
    park_hours = park.getTodayParkHours()

    today = park_hours['park_open'].strftime('%Y-%m-%d')
    park_open = park_hours['park_open'].strftime('%I:%M %p') if park_hours['park_open'] is not None else 'None'
    park_close= park_hours['park_close'].strftime('%I:%M %p') if park_hours['park_close'] is not None else 'None'
    emh_open = park_hours['emh_open'].strftime('%I:%M %p') if park_hours['emh_open'] is not None else 'None'
    emh_close = park_hours['emh_close'].strftime('%I:%M %p') if park_hours['emh_close'] is not None else 'None'
    # special_open = park_hours['special_open'].strftime('%I:%M %p') if park_hours['special_open'] is not None else 'None'
    # special_close = park_hours['special_close'].strftime('%I:%M %p') if park_hours['special_close'] is not None else 'None'
    query = "insert into DisneyDB.ParkHours (Date, ParkId, ParkOpen, ParkClose, EMHOpen, EMHClose) values ('%s', %i, '%s','%s','%s','%s')"%(today, park_db_id, park_open, park_close, emh_open, emh_close)
    cur.execute(query)
    conn.commit()


In [ ]:
#that got me the operating park hours
#now we should look at wait times per attraction grouped by park

In [4]:
import disney_parks
import pymysql
import pandas as pd
from disney_attractions import Attraction
from disney_entertainment import Entertainment

conn = pymysql.connect(host, user=username,port=port,passwd=password)
cur = conn.cursor()

park_ids = open("parkIds.txt", "r")
string = park_ids.read()
park_list = str.splitlines(string)
ids = [x.split(':')[-1] for x in park_list]

att_ids = open("attractionIds.txt", "r")
string = att_ids.read()
att_list = str.splitlines(string)
attraction_ids = [x.split(':')[-1] for x in att_list]

ent_ids = open("entertainment.txt", "r")
string = ent_ids.read()
ent_list = str.splitlines(string)
entertainment_ids = [x.split(':')[-1] for x in ent_list]

for current_id in ids:
    time = datetime.now(tz)
    date = time.date()
    time = time.time().strftime("%H:%M")
    park = disney_parks.Park(current_id)
    current_park_name = park.getParkName().replace("'","")
    print("Working On Park:" + str(current_park_name))
    query = "select * from DisneyDB.Park where Name = '" + current_park_name + "'"
    park_table = pd.read_sql_query(query, conn)
    if park_table.shape[0] < 1:
        insert_park = "insert into DisneyDB.Park (Name) values ('%s')" %(current_park_name)
        cur.execute(insert_park)
        conn.commit()
        park_table = pd.read_sql_query(query, conn)

    park_db_id = park_table['Id'][0]

    #get our wait times
    wait_times = park.getCurrentWaitTimes()
    for key, wait_dict in wait_times.items():
        is_attraction = False
        is_entertainment = False
        if key in attraction_ids:
            is_attraction = True
            #print("this is an attraction")
        elif key in entertainment_ids:
            is_entertainment = True
            #print("this is entertainment")
        else:
            print("bad_key")
            print(wait_dict)
            continue
            
        #print(key)
        #print(wait_dict)
        #check if that id exists in the ride table
        key = key.strip()
        search_query = "select * from DisneyDB.Ride where Id = " + key
        ride_table = pd.read_sql_query(search_query, conn)
        #print(ride_table.shape)
        if ride_table.shape[0] < 1:
            if is_attraction:
                #print("finding an attraction")
                current_attraction = Attraction(key)
                name = current_attraction.getAttractionName()
                #print(name)
                location = current_attraction.getAncestorLand()
            
            elif is_entertainment:
                #print("finding an entertainment")
                current_entertainment = Entertainment(key)
                name = current_entertainment.getEntertainmentName()
                #print(name)
                location = current_entertainment.getAncestorLand()

            query = "insert into DisneyDB.Ride (Id, Name, Location, ParkId) values (%i, '%s', '%s', %i)"%(int(key),name,location,park_db_id)
            #print(query)
            cur.execute(query)
            conn.commit()

            ride_table = pd.read_sql_query(search_query, conn)

        db_ride_id = ride_table["Id"][0]


        if 'wait_time' in wait_dict:
            current_wait = wait_dict['wait_time']
            query = "insert into DisneyDB.Ride_Waits (RideId, Date, Time, Wait) values (%i, '%s','%s', %i)"%(db_ride_id, date, time, current_wait)
            cur.execute(query)
            conn.commit()


Working On Park:Magic Kingdom Park
5490
136550
2219610
16124144
16414579
16491297
16491299
16512939
16767209
16767263
16767276
16767284
17272158
17511485
17511582
17823158
18466383
18485750
18677928
19062768
80010107
80010110
80010114
80010117
80010129
80010135
80010149
80010153
80010160
80010162
80010165
80010170
80010176
80010177
80010190
80010192
80010196
80010208
80010210
80010213
80010220
80010222
80010224
80010230
80010232
80069748
80069754
7922
8066
8067
8074
8075
8113
8274
8277
8279
8336
8515
81786
127313
321125
387133
15743682
15803136
15850196
16502913
16854108
16874126
17022320
17394314
17440492
17505397
17718925
17752793
18381020
18465067
18498503
18501287
18672598
Working On Park:Epcot
3831
20194
26421
56404
61525
62992
91245
107785
160914
207395
220239
16633259
17266899
17266912
17266913
17266950
17266951
17266952
17266953
17266954
17267017
17267018
17267019
bad_key
{'ride_id': '17267020', 'ride_name': 'Topiary - Winnie the Pooh', 'fast_pass_info': {'available': False}}
1

In [ ]:
siteString = "https://touringplans.com//" + "animal-kingdom" + "/attractions.json"

In [ ]:
import requests
r = requests.get(siteString)
jsonData = r.json()

In [ ]:
jsonData

In [ ]:
for item in jsonData:
    print(item['name'])

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import pymysql

locations = [{"name":"Magic Kingdom Park", "linkName":"magic-kingdom"},
            {"name":"EpCot","linkName":"epcot"},
            {"name":"Disneys Hollywood Studios","linkName":"hollywood-studios"},
            {"name":"Disneys Animal Kingdom Theme Park","linkName":"animal-kingdom"}]

conn = pymysql.connect(host, user=username,port=port,passwd=password)

cur = conn.cursor()

for location in locations:
    locationName = location["name"]
    linkName = location["linkName"]
    locationQuery = "select * from DisneyDB.Park where name = '" + locationName +"'"
    df = pd.read_sql_query(locationQuery,conn)
    if len(df.index) == 0:
        insertLocation = "insert into DisneyDB.Park (Name) values ('"+ locationName + "')"
        cur.execute(insertLocation)
        df = pd.read_sql_query(locationQuery,conn)
        conn.commit()
    locationId = df['Id'][0]
    siteString = "https://touringplans.com//" + linkName + "/attractions.json"
    r = requests.get(siteString)
    jsonData = r.json()
    #print(jsonData)
    for ride in jsonData:
        tableName = ride["name"]
        rideName = ride["permalink"]
        #tableName = tableName.replace(" ", "-")
        tableName = tableName.replace("'","")
        tableName = tableName.replace(u"\u2019","")
        tableName = tableName.replace("&", "and")
        tableName = tableName.replace(": "," - ")
        print(rideName)
        attractionString = "https://touringplans.com/"+linkName+"/attractions/"+rideName+".json"
        rideRequest = requests.get(attractionString)
        rideJson = rideRequest.json()
        #print(rideJson["name"] + " opened on " + str(rideJson["opened_on"]))
        rideQuery = "select * from DisneyDB.Ride where name = '"+ tableName + "'"
        rideTable = pd.read_sql_query(rideQuery, conn)
        if len(rideTable.index) == 0:
            insertRide = "insert into DisneyDB.Ride (Name, OpeningDate, Tier, ParkId) values ('" + tableName + "','"+ str(rideJson["opened_on"]) + "','"+ str(rideJson["scope_and_scale_code"])+ "',"+str(locationId)+")"
        else:
            rideId = rideTable['Id'][0]
            insertRide = "update DisneyDB.Ride set OpeningDate='"+ str(rideJson["opened_on"]) + "', Tier = '"+ str(rideJson["scope_and_scale_code"])+ "' where Id = "+ str(rideId)
        cur.execute(insertRide)
        conn.commit()
